# Board Game Recommender_Contend Based

https://www.kaggle.com/extralime/20000-boardgames-dataset  
데이터를 기반으로, 보드게임을 추천하는 프로젝트를 만들려고 한다.  
평점 데이터가 존재하지는 않아, 데이터를 정리하여 특성을 추출하고, 새로운 보드게임을 추천해주자.

## 사용 모델

추천 시스템은 크게, CB와 CF, Hybrid로 나누어진다. 딥러닝등을 쓰는 방법도 존재하나, 주어진 데이터셋에서는 Item의 특성만 담겨 있으므로 적합한 모델은 Contend-Based Model이다.

## Content-Based란?
Contend-Based 추천의 경우, Item의 특성에 따른 유사도에 따라 비슷한 Item을 추천해 주는 방식이다.

## 만들고자 하는 Recommender
가장 좋아하는 3개의 보드게임을 입력하면, 그와 유사한 혹은 새로 해볼만한 보드게임 10개를 추천하고자 한다.
이 때, 플레이 인원 또한 입력받아, 플레이 인원에 맞는 보드게임을 추천하게하고자 한다.

## Data 및 Package 준비

In [1]:
import numpy as np
import pandas as pd
data = pd.read_csv('C:/Users/HP/Desktop/Recommender_Project/boardgames1.csv')

In [2]:
data.head()

,objectid,name,yearpublished,sortindex,minplayers,maxplayers,minplaytime,maxplaytime,minage,min_community,...,boardgameartist,boardgamepublisher,boardgamehonor,boardgamecategory,boardgameversion,boardgamemechanic,boardgameexpansion,boardgamefamily,description,gamelink
0,174430,Gloomhaven,2017,1,1,4,60,120,12,3.0,...,"['Alexandr Elichev', 'Josh T. McDowell', 'Alva...","['Cephalofair Games', 'Albi', 'Asmodee', 'Feue...",['2017 Best Science Fiction or Fantasy Board G...,"['Adventure', 'Exploration', 'Fantasy', 'Fight...","['Chinese edition', 'Czech edition', 'English ...","['Campaign / Battle Card Driven', 'Cooperative...","['Gloomhaven: Forgotten Circles', 'Gloomhaven:...","['Campaign Games', 'Components: Miniatures', '...",Gloomhaven is a game of Euro-inspired tactica...,/boardgame/174430/gloomhaven
1,161936,Pandemic Legacy Season 1,2015,2,2,4,60,60,13,4.0,...,['Chris Quilliams'],"['Z-Man Games', 'Asterion Press', 'Devir', 'Fi...",['2015 Cardboard Republic Immersionist Laurel ...,"['Environmental', 'Medical']","['Chinese blue edition', 'Chinese red edition'...","['Action Points', 'Cooperative Game', 'Hand Ma...",['None'],"['Campaign Games', 'Legacy', 'Pandemic']",Pandemic Legacy is a co-operative campaign gam...,/boardgame/161936/pandemic-legacy-season-1
2,167791,Terraforming Mars,2016,3,1,5,120,120,12,3.0,...,['Isaac Fryxelius'],"['FryxGames', 'Arclight', 'Fantasmagoria', 'Gh...",['2016 Cardboard Republic Architect Laurel Nom...,"['Economic', 'Environmental', 'Industry / Manu...","['Bulgarian edition', 'Chinese edition', 'Czec...","['Card Drafting', 'End Game Bonuses', 'Hand Ma...",['French Championship Promo Cards (fan expansi...,"['Fryxgames Future Timeline', 'Planets: Mars',...","In the 2400s, mankind begins to terraform the ...",/boardgame/167791/terraforming-mars
3,182028,Through the Ages A New Story of Civilization,2015,4,2,4,120,120,14,3.0,...,"['Filip Murmak', 'Radim Pech', 'Jakub Politzer...","['Czech Games Edition', 'Cranio Creations', 'D...",['2015 Golden Geek Best Strategy Board Game No...,"['Card Game', 'Civilization', 'Economic']","['Chinese edition', 'Czech edition', 'English ...","['Action Points', 'Auction/Bidding', 'Auction:...",['Through the Ages: New Leaders and Wonders'],"['Tableau Building', 'Through the Ages']",Through the Ages: A New Story of Civilization ...,/boardgame/182028/through-ages-new-story-civil...
4,224517,Brass Birmingham,2018,5,2,4,60,120,14,3.0,...,"['Lina Cossette', 'David Forest', 'Damien Mamm...","['Roxley', 'BoardM Factory', 'Conclave Editora...",['2018 Golden Geek Best Board Game Artwork & P...,"['Economic', 'Industry / Manufacturing', 'Tran...","['English deluxe edition', 'English retail edi...","['Hand Management', 'Income', 'Loans', 'Market...",['None'],"['Beer', 'Brass', 'Cities: Birmingham (England...",Brass: Birmingham is an economic strategy game...,/boardgame/224517/brass-birmingham


Kaggle 링크에는, 이 data에 대한 설명이 있다. 이를 가져오면 다음과 같다.

- objectid - the identifier on boardgamegeeks.com
- name - name of the game
- yearpublished - the year the game was published
- sortindex - rank of the game on bbg.com
- minplayer - minumum number of players per the publishers
- maxplayers - maximum number of players per the publishers
- minplaytime - minimum playtime required per the publishers
- maxplaytime - maximum playtime per the publishers
- minage - minimum age requiremnet per the publishers
- min_community - minimum players per the community
- max_community - max players per the community
- totalvotes - total number of community vote
- playerage - minimum age requirement per the community
- languagedependence - a rank of amount of in-game text is required during game play
    - '(no votes)':0,
    - 'Extensive use of text - massive conversion needed to be playable':4,
    - 'Moderate in-game text - needs crib sheet or paste ups':3,
    - 'No necessary in-game text':1,
    - 'Some necessary text - easily memorized or small crib sheet':2,
    - 'Unplayable in another language':5    
- userrated - number of users that have rated the game
- average - user average rating from 1-10
- baverage - from the site, determined to be an anti-skewing effort by bgg with system added mid range ratings (from 1-10)
- stddev - average standard deviation of a rating
- avgweight - a complexity rating from 1-5 Weight
- numweights - number of weight votes
- numgeeklists - number of geeks with game on list
- numtrading - number of people trading the game
- numwanting - number of people wanting the game
- numcomments - number of comments on the site on this game
- siteviews - number of views on the site
- numplays - number of times game was played (according to site users?)
- numplays_month - number of plays per month
- news - number news articles on game
- blogs - number of blogs regarding game
- weblink - number of weblinks for the game
- podcast - number of podcasts on the game
- label - category of game (mostly boardgame)
- boardgamedesigner_cnt - count of designers
- boardgameartist_cnt - artist count
- boardgamepublisher_cnt - publisher count
- boardgamehonor_cnt - awards count
- boardgamecategory_cnt - category count
- boardgamemechanic_cnt - game mechanics count
- boardgameexpansion_cnt - expansion count
- boardgameversion_cnt - version count (languages)
- boardgamefamily_cnt - game family count
- boardgamedesigner - list of game designers
- boardgameartist - list of game artists
- boardgamepublisher - list of publishers
- boardgamehonor - list of awards
- boardgamecategory - list of categories
- boardgameversion - list of versions
- boardgamemechanic - a list of mechanics
- boardgameexpansion - a list of expansions
- boardgamefamily - a list of boardgames family
- description - full text description of game
- gamelink - a link to the game on bgg

모든 변수를 사용하진 않을 것이고, 아이템의 '특성'에 초점을 주려고 한다.  
보드게임에서 생각해볼 수 있는 특성은 다음과 같다.
  - 게임의 난이도
  - 게임이 가지고 있는 테마
  - 플레이 시간
  - 플레이 인원수
  - 게임이 돌아가는 원리(매커니즘)
  - 언어 의존도
  - ETC...

여기서는 난이도, 테마, 인원수, 플레이 시간, 매커니즘, 언어의존도, 보드게임작가 및 평점 데이터만을 사용하려고 한다. 이를 위해 변수 순서를 다시 지정하면 다음과 같다.

In [3]:
data = data[['name', 'sortindex', 'minplayers', 'maxplayers', 
             'minplaytime', 'maxplaytime', 'languagedependence',
            'average', 'usersrated', 'avgweight',
            'boardgamecategory', 'boardgamemechanic']]

In [4]:
data.head(5)

,name,sortindex,minplayers,maxplayers,minplaytime,maxplaytime,languagedependence,average,usersrated,avgweight,boardgamecategory,boardgamemechanic
0,Gloomhaven,1,1,4,60,120,4,8.85292,31254,3.8078,"['Adventure', 'Exploration', 'Fantasy', 'Fight...","['Campaign / Battle Card Driven', 'Cooperative..."
1,Pandemic Legacy Season 1,2,2,4,60,60,4,8.62499,34729,2.8301,"['Environmental', 'Medical']","['Action Points', 'Cooperative Game', 'Hand Ma..."
2,Terraforming Mars,3,1,5,120,120,3,8.42299,48339,3.2313,"['Economic', 'Environmental', 'Industry / Manu...","['Card Drafting', 'End Game Bonuses', 'Hand Ma..."
3,Through the Ages A New Story of Civilization,4,2,4,120,120,4,8.49419,18269,4.3850,"['Card Game', 'Civilization', 'Economic']","['Action Points', 'Auction/Bidding', 'Auction:..."
4,Brass Birmingham,5,2,4,60,120,1,8.62031,10070,3.9122,"['Economic', 'Industry / Manufacturing', 'Tran...","['Hand Management', 'Income', 'Loans', 'Market..."


2만개의 데이터가 있지만 모든 데이터를 다 사용하기는 어려울 것이다. 그 이유는 해당 데이터의 경우는 user들의 직접적인 평가를 기반으로 하고 있다. 따라서, usersrated가 일정 값 이하인 경우, 충분한 데이터가 모였다고 하기는 어려울 것이다. 

In [5]:
data['usersrated'].describe()

count    20000.000000
mean       739.616350
std       3096.843206
min          0.000000
25%         47.000000
50%        105.000000
75%        333.250000
max      90730.000000
Name: usersrated, dtype: float64

2만개중 절반 가량의 보드게임은 100표도 얻지 못했음을 알 수 있다. 또한 평균은 1,000건도 되지 않음을 알 수 있다. 그러나 3사분위수가 300건대인것을 보면 일정 보드게임들에 대해서만 '유의미한' 관심을 받았음을 알 수 있다. 여기서는 1,000건 이상의 평가를 받은 보드게임에 대해서만 추천을 실시하고자 한다.

In [6]:
data = data.loc[data['usersrated']>=1000,:]

In [7]:
len(data)

2347

그 결과, 전체 20000개 중 2347개의 데이터를 추출하였다. 이를 바탕으로 Content-Based를 하고자 한다.

In [8]:
data.reset_index(drop = True, inplace = True)

## category, mechniac 대하여

두 변수의 유사도 측정을 위해 pearson_corr를 활용한다.

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

category_list = [''.join(x) for x in data.boardgamecategory]
enc = CountVectorizer()
category_arr = enc.fit_transform(category_list).toarray()
np.shape(category_arr)

(2347, 200)

In [10]:
from scipy.stats.stats import pearsonr 
maxx = -1
max_idx = -1
for i in range (1, 2347):
    if maxx < pearsonr(category_arr[0], category_arr[i])[0]:
        maxx = pearsonr(category_arr[0], category_arr[i])[0]
        max_idx = i
print(data['boardgamecategory'][max_idx])
print(data['boardgamecategory'][0])
print(data['name'][max_idx])

['Adventure', 'Exploration', 'Fantasy', 'Fighting', 'Miniatures']
['Adventure', 'Exploration', 'Fantasy', 'Fighting', 'Miniatures']
Descent  Journeys in the Dark (Second Edition)


In [11]:
mechniac_list = [''.join(x) for x in data.boardgamemechanic]
enc = CountVectorizer()
mechniac_arr = enc.fit_transform(mechniac_list).toarray()
np.shape(mechniac_arr)

(2347, 279)

In [12]:
from scipy.stats.stats import pearsonr 
maxx = -1
max_idx = -1

for i in range (1, 2347):
    if maxx < pearsonr(mechniac_arr[0], mechniac_arr[i])[0]:
        maxx = pearsonr(mechniac_arr[0], mechniac_arr[i])[0]
        max_idx = i
print(data['boardgamemechanic'][max_idx])
print(data['boardgamemechanic'][0])
print(data['name'][max_idx])
print(max)

['Area Movement', 'Campaign / Battle Card Driven', 'Cooperative Game', 'Dice Rolling', 'Hand Management', 'Modular Board']
['Campaign / Battle Card Driven', 'Cooperative Game', 'Grid Movement', 'Hand Management', 'Legacy Game', 'Modular Board']
Gears of War  The Board Game
<built-in function max>


각 행렬을 얻었고, 0번 아이템에 대해 유사도가 가장 높은 보드게임을 뽑아보았다.

## 실제 추천 만들기

우리의 목표를 다시 생각해보자.

가장 좋아하는 3개의 보드게임을 입력하면, 그와 유사한 혹은 새로 해볼만한 보드게임 10개를 추천하고자 한다.
이 때, 플레이 인원 또한 입력받아, 플레이 인원에 맞는 보드게임을 추천하게하고자 한다.

다음과 같은 방식으로 구현하고자 한다.
3개의 게임에 대해 동일한 가치를 부여하고,
Mechniac이랑 Category 또한 동일한 weight로 계산하고자 한다.

이를 바탕으로 우선은 유사도가 가장 높은 게임 100개를 평점 순으로 정렬하려고 한다.
그 후 플레이 인원수가 맞는지 확인 후,
맨 마지막으로 weight가 선호게임 대비 높은 경우(해본 게임 대비) 흥미도가 낮을 수 있으므로
3개의 베스트 보드게임 중 weight 가장 낮은거의 -0.5 ~ 가장 높은 거의 +0.5  범주안에 들 경우 추천 대상에 넣는다.

In [25]:
board_title = list(data.name)
print('number of people')
people = int(input())

while True:
    print('first input')
    first = str(input())
    if first in board_title :
        break
        
while True:
    print('second input')
    second = str(input())
    if second in board_title  and first != second:
        break
        
while True:
    print('third input')
    third = str(input())
    if third in board_title  and third not in [first, second]:
        break
        
idx1 = board_title.index(first)
idx2 = board_title.index(second)
idx3 = board_title.index(third)
level_1 = data.avgweight[idx1]
level_2 = data.avgweight[idx2]
level_3 = data.avgweight[idx3]

low_level = min(level_1, level_2, level_3) - 0.5
high_level = max(level_1, level_2, level_3) + 0.5

top_list = [idx1, idx2, idx3]
score_list = []

for i in range (0, 2347):
    if i in top_list:
        score_list.append((0, i))
    else:
        score = 0
        score += (pearsonr(category_arr[idx1], category_arr[i])[0])
        score += (pearsonr(category_arr[idx2], category_arr[i])[0])
        score += (pearsonr(category_arr[idx3], category_arr[i])[0])
        score += (pearsonr(mechniac_arr[idx1], mechniac_arr[i])[0])
        score += (pearsonr(mechniac_arr[idx2], mechniac_arr[i])[0])
        score += (pearsonr(mechniac_arr[idx3], mechniac_arr[i])[0])
        score_list.append((score, i))

score_list.sort(reverse = True)
score_list = sorted(score_list[:100], key = lambda x: x[1])

recom_list = []
for item in score_list:
    idx = item[1]
    if data.minplayers[idx] <= people <= data.maxplayers[idx]:
        if low_level <= data.avgweight[idx] <= high_level:
            recom_list.append(data.name[idx])
    if len(recom_list) == 10:
        break
print(recom_list)

number of people
2
first input
Lost Cities
second input
PatchWork
second input
Patchwork
third input
Codenames  Duet
['Codenames XXL', 'Jaipur', 'Sushi Go Party!', 'Arboretum', 'Codenames  Pictures XXL', 'Hanabi Deluxe', 'The Grizzled', 'Schotten Totten', 'Paperback', '\\u05e1\\u05d5\\u05e9\\u05d9 Go!']


### 예시 1
철저하게 내 취향대로 해보면  
number of people    
4     

first input  
Race for the Galaxy  
second input  
Mombasa  
third input  
Trajan  
['Twilight Imperium (Fourth Edition)', 'Through the Ages  A Story of Civilization', 'Roll for the Galaxy', 'Kemet', 'Grand Austria Hotel', 'La Granja', 'Imperial Settlers', 'Glory to Rome', 'Imperial', "Aeon's End  War Eternal"]

실제로, 롤 포더 갤럭시와 레이스 포 더 갤럭시는 굉장히 유사한 게임성을 가졌기에 추천이 될 만한 게임이다.
그 외에도 주사위 기반의 전략게임인 그랜드 오스트리아 호텔이나 라그랑하도 많이 추천받은 게임으로 어느정도는 적합해보인다.

### 예시 2
초보자용 게임을 몇개 넣어보았다.  
number of people  
4  
first input  
BANG!  
second input  
The Great Dalmuti  
third input  
Halli Galli  
['Love Letter', 'Skull & Roses Red', '\\u30af\\u30fc', 'Love Letter Premium', 'Love Letter  Batman', 'Red7', 'Cribbage', 'Wizard', 'Diamonds  Second Edition', 'Shadow Hunters']

어느정도 중복이 조금 보이긴 하지만, 러브레터라는 초보자용으로 매우 적합하고 훌륭한 게임을 얻게 된다.

### 예시 3
커플 보드게임 추천을 해보자.  
number of people  
2  
first input  
Lost Cities  
second input  
Patchwork  
third input  
Codenames  Duet  
['Codenames XXL', 'Jaipur', 'Sushi Go Party!', 'Arboretum', 'Codenames  Pictures XXL', 'Hanabi Deluxe', 'The Grizzled', 'Schotten Totten', 'Paperback', '\\u05e1\\u05d5\\u05e9\\u05d9 Go!']

비슷한 코드네임 시리즈와 함께 커플이서 해볼만한 협동게임인 하나비, 2인 머리쓰는 게임으로 적합한 자이푸르 등이 추천되는 것을 볼 수 있다.

## 더 좋은 추천을 위해서는?
사실 이번 데이터셋에서는 user의 평점은 없었고, 보드게임의 특성만이 있었다.
준비된 데이터셋에서는 한글 관련 문제인지 글자깨짐 현상도 발생했고, 특히 보드게임의 매커니즘이나 테마에서 너무 세부분류가 많아 잘 정돈되어 있지 않음을 느꼈다.  조금 더 정밀한 추천을 위해서는 세부분류를 통합해야 하고, 아울러 user의 평점 데이터를 바탕으로 CF를 해야 조금 더 적합한 추천이 가능할 것이다.